In [4]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np

In [42]:
schedule = nfl.import_schedules([2023])
data = nfl.import_pbp_data([2023])

2023 done.
Downcasting floats.


In [104]:
class GameData:
    def __init__(self, team, game):
        self.team = team
        self.stats = {}
        self.game = game
        self.id = game['game_id'][0]

        self.is_home = team == game['home_team'][0]
        self.stats['team'] = team
        self.stats['id'] = self.id
        self.initialize_stats()

    
    def initialize_stats(self):
        winner = self.game['home_team'][0] if self.game['home_score'][0] > self.game['away_score'][0] else self.game['away_team'][0]
        self.stats['won_game'] = winner == self.team
        self.stats['points'] = self.game['home_score'][0] if self.is_home else self.game['away_score'][0]
        self.stats['opp_points'] = self.game['home_score'][0] if not self.is_home else self.game['away_score'][0]
        for index, play in self.game.iterrows():
            self.add_stats_from_play(play)

    def add_stats_from_play(self, play):
        team = play['possession_team']

        if team == self.team:
            self.add_offensive_stats(play)
        else:
            self.add_defensive_stats(play)
    
    def add_offensive_stats(self, play):
        yards_gained = play['yards_gained']
        play_type = play['play_type']

        if not np.isnan(yards_gained):
            self.stats['yards'] = self.stats.get('yards', 0) + yards_gained

        if play_type == 'field_goal':
            self.stats['field_goals_attempted'] = self.stats.get('field_goals_attempted', 0) + 1
            made = play['field_goal_result'] == 'made'
            self.stats['field_goals_made'] = self.stats.get('field_goals_made', 0) + made
        
        if play['interception'] == 1 or play['fumble_lost'] == 1:
            self.stats['offensive_turnovers'] = self.stats.get('offensive_turnovers', 0) + 1

    def add_defensive_stats(self, play):
        yards_gained = play['yards_gained']
        play_type = play['play_type']

        if not np.isnan(yards_gained):
            self.stats['opp_yards'] = self.stats.get('opp_yards', 0) + yards_gained

        if play['interception'] == 1 or play['fumble_lost'] == 1:
            self.stats['defensive_turnovers'] = self.stats.get('defensive_turnovers', 0) + 1

        # [None, 'kickoff', 'run', 'pass', 'punt', 'no_play', 'extra_point', 'field_goal', 'qb_kneel', 'qb_spike']
    
    def __str__(self):
        return self.id
    
    def __repr__(self):
        return self.id

In [105]:
game_data_arr = []

for game_id in data['game_id'].unique():
    if schedule[schedule['game_id'] == game_id]['game_type'].values[0] == 'REG':
        if len(game['possession_team'].unique()) == 0:
            # this means the game doesn't have PBP data yet
            continue

        game = data[data['game_id'] == game_id].reset_index()

        home_team = game['home_team'][0]
        away_team = game['away_team'][0]

        home_game = GameData(home_team, game)
        away_game = GameData(away_team, game)
        game_data_arr.append(home_game)
        game_data_arr.append(away_game)

In [106]:
df[df['team'] == 'ARI']

,team,id,won_game,points,opp_points,opp_yards,yards,offensive_turnovers,field_goals_attempted,field_goals_made,defensive_turnovers
1,ARI,2023_01_ARI_WAS,False,16,20,248.0,210.0,2.0,3.0,3.0,3.0
54,ARI,2023_02_NYG_ARI,False,28,31,439.0,381.0,NaN,3.0,2.0,1.0
72,ARI,2023_03_DAL_ARI,True,28,16,416.0,400.0,NaN,3.0,3.0,1.0
97,ARI,2023_04_ARI_SF,False,16,35,395.0,362.0,NaN,1.0,1.0,NaN
134,ARI,2023_05_CIN_ARI,False,20,34,380.0,294.0,3.0,NaN,NaN,1.0
157,ARI,2023_06_ARI_LA,False,9,26,382.0,345.0,3.0,3.0,3.0,NaN
187,ARI,2023_07_ARI_SEA,False,10,20,318.0,249.0,1.0,2.0,1.0,2.0
214,ARI,2023_08_BAL_ARI,False,24,31,268.0,312.0,2.0,1.0,1.0,NaN
245,ARI,2023_09_ARI_CLE,False,0,27,326.0,58.0,3.0,NaN,NaN,NaN
272,ARI,2023_10_ATL_ARI,True,25,23,254.0,352.0,1.0,4.0,4.0,NaN


In [108]:
df = pd.DataFrame([game.stats for game in game_data_arr])
df = df.drop(columns=['id'])

aggregated = df.groupby('team').mean()
aggregated

,won_game,points,opp_points,opp_yards,yards,offensive_turnovers,field_goals_attempted,field_goals_made,defensive_turnovers
team,,,,,,,,,
ARI,0.235294,19.411765,26.764706,427.764706,306.714286,2.000000,2.166667,1.916667,1.700000
ATL,0.411765,18.882353,21.941176,386.000000,327.500000,1.909091,2.333333,2.083333,2.090909
BAL,0.764706,28.411765,16.470588,343.647059,372.333333,1.900000,2.571429,2.214286,2.384615
BUF,0.647059,26.529412,18.294118,323.764706,380.687500,2.076923,2.076923,1.692308,2.214286
CAR,0.117647,13.882353,24.470588,336.352941,271.214286,2.000000,2.000000,1.714286,1.300000
CHI,0.411765,21.176471,22.294118,385.823529,318.071429,2.400000,2.416667,2.250000,2.416667
CIN,0.529412,21.529412,22.588235,409.705882,322.200000,1.333333,1.933333,1.600000,2.000000
CLE,0.647059,23.294118,21.294118,309.470588,337.066667,2.428571,2.466667,2.266667,2.384615
DAL,0.705882,29.941176,18.529412,345.647059,369.733333,1.625000,2.357143,2.357143,2.071429
